In [ ]:
%matplotlib inline
from __future__ import division 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_style('whitegrid', {'axes.grid':  False})
from PIL import Image
import sys
sys.path.append('/home/philipp/caffe/python') 
import caffe

caffe.set_mode_cpu()

#load the model
model_path = '/home/philipp/caffe/models/bvlc_reference_caffenet/'
net_fn = model_path + 'deploy.prototxt'
param_fn = model_path+ 'caffenet_train_iter_10000_dropout06.caffemodel'

net = caffe.Net(net_fn,
                param_fn,
                caffe.TEST)

# load input and configure preprocessing
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer_path ='/home/philipp/caffe/python/caffe/imagenet/'
transformer.set_mean('data', np.load(transformer_path+'ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

#note we can change the batch size on-the-fly
#since we classify only one image, we change batch size from 10 to 1
net.blobs['data'].reshape(1,3,227,227)

image_path = '/home/philipp/caffe/data/AVA/trainingset/preprocesed/'
all_image_path = image_path+'*.jpg'


In [ ]:
def return_class(image):
#load the image in the data layer
    im = caffe.io.load_image(image)
    net.blobs['data'].data[...] = transformer.preprocess('data', im)
    out = net.forward()
    return out['prob'].argmax()

##### Example Output for single image

In [ ]:
image = image_path+'14678.jpg'
array = return_class(image)
print 'This  image is :',array
plt.imshow(caffe.io.load_image(image));

In [ ]:
import os
print 'number of image files: ', len(os.walk(image_path).next()[2])

In [ ]:
import glob
# Create dict with image name as key and the prediction from the NN as value
lst_pred = {}
for row_number, image in enumerate(glob.glob(all_image_path)): #fill an entire row
    pred = return_class(image) 
    lst_pred[image.split('/')[8]]= pred

In [ ]:
import pandas as pd
df_NN_pred = pd.DataFrame.from_dict(lst_pred, orient='index')

path_train_label= '/home/philipp/caffe/data/AVA/train.txt'

data = pd.read_csv(path_train_label, sep=" ", header = None)
data.columns = ["image", 'label']
data = data.set_index('image')
data= data.join(df_NN_pred, how='outer')
data = data.rename(columns={0: 'pred'})
data.head()

#### Model Fit on training data

In [ ]:
data['fit'] = abs(data.label-data.pred)
data.fit[data.fit==0].count()/len(data)

In [ ]:
data.to_csv('ModelFit_on_TrainingData_v2.csv')